In [ ]:
from pandas import read_csv,Series,DataFrame
from numpy import sign,NaN,where,sqrt,mean
from pandas.core.indexes.datetimes import DatetimeIndex
import swifter

# data

In [ ]:
macro=read_csv('data.csv',index_col=[0],parse_dates=[0])
macro.columns=['US','CN','UK','FR','DE','CA','AU','NZ','EU','JP','IT','CH','UAE','IN','NO','SE','ZA']

In [ ]:
mkt=read_csv('BloombergCommos.csv',index_col=[0],parse_dates=[0])[1:].sort_index().astype(float).pct_change().dropna()
mkt.columns=['SoybeanOil','Corn','WTI','Brent',
'Cotton','Copper','HeatingOil','Coffee',
'HRWWheat','Aluminium','LiveCattle','LeanHogs',
'Lead','Nickel','Zinc','NaturalGas',
'Gasoil','Soybean','Sugar','SoybeanMeal',
'Wheat','Gasoline','Gold','Silver']

In [ ]:
fees={'SoybeanOil':.0003,'Corn':.0003,'WTI':.0003,'Brent':.0003,
'Cotton':.0003,'Copper':.0003,'HeatingOil':.0003,'Coffee':.0003,
'HRWWheat':.0003,'Aluminium':.0003,'LiveCattle':.0003,'LeanHogs':.0003,
'Lead':.0003,'Nickel':.0003,'Zinc':.0003,'NaturalGas':.0003,
'Gasoil':.0003,'Soybean':.0003,'Sugar':.0003,'SoybeanMeal':.0003,
'Wheat':.0003,'Gasoline':.0003,'Gold':.0003,'Silver':.0003}

# functions

In [ ]:
def AdjustSignal(signal,limit=.1):
    newSignal=signal.copy()
    for i in range(1,len(signal)):
        x=signal[i]-newSignal[i-1]
        newSignal[i]=newSignal[i-1]+min([abs(x),limit])*sign(x)
    return newSignal

In [ ]:
def perfs(signals):
    strat=signals.mul(mkt[signals.columns],axis=0).sum(1).reindex(signals.index)
    for x in signals.columns:
        strat-=signals[x].diff().abs()*fees[x]
    return strat

In [ ]:
def avgSpace(row,eps,inputs,outputs,df):
    x,y=row[inputs[0]],row[inputs[1]]
    mask=(df[inputs[0]]>=(row[inputs[0]]-eps[0])) & (df[inputs[0]]<=row[inputs[0]]+eps[0])
    mask &= (df[inputs[1]]>=(row[inputs[1]]-eps[1])) & (df[inputs[1]]<=row[inputs[1]]+eps[1])
    tmp=df[mask][outputs].mean()
    return tmp
def sharpeSpace(row,eps,inputs,outputs,df):
    x,y=row[inputs[0]],row[inputs[1]]
    mask=(df[inputs[0]]>=(row[inputs[0]]-eps[0])) & (df[inputs[0]]<=row[inputs[0]]+eps[0])
    mask &= (df[inputs[1]]>=(row[inputs[1]]-eps[1])) & (df[inputs[1]]<=row[inputs[1]]+eps[1])
    tmp=df[mask][outputs]
    tmp=tmp.mean()/tmp.std()
    return tmp

In [ ]:
from plotly.graph_objects import Figure, Scatter
def chart(strat,title,bench=None,width=1000,height=600,outsample=None,stratName='Strategy',benchName='Benchmark',save=None):
    fig=Figure()
    if type(bench)==Series or type(bench)==DataFrame:
        d=strat.to_frame('Strat').join(bench.rename('Benchmark'),how='left').fillna(0)
    else:
        d=strat.to_frame('Strat').fillna(0)
    # d=d[d.index>='2016-01-01']
    d=d[d.index>=strat.replace(0,NaN).dropna().index.min()]
    fig.add_trace(Scatter(x=d.index,y=d.Strat.cumsum(),mode='lines',line=dict(width=2.5,color='gold'),name=stratName))
    if type(bench)==Series or type(bench)==DataFrame:
        fig.add_trace(Scatter(x=d.index,y=d.Benchmark.cumsum(),mode='lines',line=dict(width=2.5,color='white'),name=benchName))
    annotations=[]
    if type(outsample)==DatetimeIndex:
        annotations.append(dict(xref='paper', yref='paper', x=0.5, y=1,xanchor='center', yanchor='bottom',text= title+' : '+str(round(d.Strat.mean()/d.Strat.std()*sqrt(252),2))+' ('+str(round(d.Strat.reindex(outsample).mean()/d.Strat.reindex(outsample).std()*sqrt(252),2))+')',font=dict(family='Arial',size=30,color='white'),showarrow=False))
    else:
        annotations.append(dict(xref='paper', yref='paper', x=0.5, y=1,xanchor='center', yanchor='bottom',text= title+' : '+str(round(d.Strat.mean()/d.Strat.std()*sqrt(252),2)),font=dict(family='Arial',size=30,color='white'),showarrow=False))
    fig.update_layout(annotations=annotations,autosize=False,width=width,height=height,font=dict(size=16,color="white"),legend=dict(y=-0.17,font=dict(family='Arial',size=18,color='white'),orientation='h'),paper_bgcolor='rgb(30,30,30)',plot_bgcolor='rgb(30,30,30)')
    fig.update_layout(yaxis_tickformat='.0%')
    fig.update_xaxes(color='white',showgrid=False,tickangle=0,tickfont_size=18)
    fig.update_yaxes(color='white',gridcolor='grey',tickfont_size=18,zerolinewidth=0,zerolinecolor="grey",title='Cumulative Returns')
    if type(outsample)==DatetimeIndex:
        fig.add_vrect(x0=outsample.min(),x1=outsample.max(),fillcolor='gold',layer='below',opacity=.25,line_width=0)
    if save!=None:
        fig.write_image(save)
    fig.show()

In [ ]:
from sklearn.model_selection import train_test_split
from pickle import dump
def Trend(indicator,assetsW,lag,window,limit=None):
    """generates signals

    Args:
        indicator (DataFrame): indicators used
        assetsW (DataFrame): assets on which we apply signals
        lag (int): lag
        window (int): window
        limit (float, optional): Max Turnover per day. Defaults to None.

    Returns:
        DataFrame: _description_
    """
    sg=indicator.diff(window).apply(sign).reindex(assetsW.index).ffill().shift(1+lag)
    if limit==None:
        return assetsW.mul(sg,axis=0)
    else:
        return assetsW.mul(AdjustSignal(sg,limit=limit),axis=0)
def GSTrend(indicator,assetsW,title,lags=range(1,20),windows=range(1,25,1),eps=[2,1],target='sharpeCV',outsample='2021-01-01',limit=None,smooth=avgSpace,cvType=min):
    """Creates grid search and smoothing of grid search to find optimal parameters on insample with cross validation. Then implements strategy on complete dataset and creates chart

    Args:
        indicator (DataFrame): Indicator
        assetsW (DataFrame): Assets on which we implement the strategy
        title (str): Name of strategy
        lags (range, optional): Lags to test in grid search. Defaults to range(1,20).
        windows (range, optional): Windows to test in grid search. Defaults to range(1,25,1).
        eps (list, optional): range of values for smoothing. Defaults to [2,10].
        target (str, optional): metric used to find best parameters. Defaults to 'sharpeCV'.
        outsample (str, optional): Date when the outsample starts. Defaults to '2021-01-01'.
        limit (float, optional): maximum turnover daily. Defaults to None.
        smooth (function, optional): function used to smooth the grid search (either avgSpace or sharpeSpace). Defaults to avgSpace.
        cvType (function, optional): function used for cross validation (either min or mean). Defaults to min.
    """
    assetsWTR=assetsW[assetsW.index<outsample]
    tr,te=train_test_split(assetsWTR.index,test_size=0.5,shuffle=False)
    res=[]
    for window in windows:
        if limit==None:
            d=indicator.diff(window).apply(sign).reindex(assetsWTR.index).ffill().dropna()
        else:
            d=AdjustSignal(indicator.diff(window).apply(sign).reindex(assetsWTR.index).ffill().dropna(),limit=limit)
        for lag in lags:
            sg=d.shift(1+lag)
            strat=perfs(assetsWTR.mul(sg,axis=0))
            stratTR=strat.reindex(tr)
            stratTE=strat.reindex(te)
            sh=strat.mean()/strat.std()*sqrt(252)
            shTR=stratTR.mean()/stratTR.std()*sqrt(252)
            shTE=stratTE.mean()/stratTE.std()*sqrt(252)
            pr=(1+strat).cumprod()
            acc=(perfs(assetsWTR).apply(sign)*strat.apply(sign)).clip(0,1)
            accTR=acc.reindex(tr)
            accTE=acc.reindex(te)
            res.append([lag,window,sh,cvType([shTR,shTE]),acc.mean(),cvType([accTR.mean(),accTE.mean()]),
            (pr/pr.cummax()-1).min()])
    res=DataFrame(res,columns=['lag','window','sharpe','sharpeCV','accuracy','accuracyCV','maxDD'])
    # dump(res,open('GridSearch/'+title+'.p','wb'))
    res=res[['lag','window']].join(res.swifter.apply(lambda x: smooth(x,eps,['lag','window'],['sharpe','sharpeCV','accuracy','accuracyCV','maxDD'],res),axis=1))
    bestLag,bestWindow=res.sort_values(by=target,ascending=False)[['lag','window']].iloc[0,:2]
    sg=Trend(indicator,assetsW,bestLag,bestWindow)
    strat=perfs(sg)
    # chart(strat=strat,title=title+' lag '+str(bestLag)+' window '+str(bestWindow)+' : ',bench=perfs(assetsW).rename('Bench'),width=1000,height=600,outsample=bbg[bbg.index>=outsample].index,save='Charts/'+title+'.png')
    chart(strat=strat,title=title+' lag '+str(bestLag)+' window '+str(bestWindow)+' : ',bench=perfs(assetsW).rename('Bench'),width=1000,height=600,outsample=bbg[bbg.index>=outsample].index)
    # dump(sg,open('Weights/'+title+'.p','wb'))
    # dump(strat,open('Returns/'+title+'.p','wb'))

In [ ]:
def createAssets(long,short=None):
    if long!=None:
        temp=DataFrame(index=bbg.index,columns=[long])
        temp[long]=1
        if short!=None:
            temp[short]=-1
    else:
        temp=DataFrame(index=bbg.index,columns=[short])
        temp[short]=-1
    return temp
def createIndicator(db,long):
    return db[long]

In [ ]:
def createStrat(title,dbIndic,longIndicator,longAsset,shortAsset=None,lags=range(1,20),windows=range(1,25,1),eps=[2,1],target='sharpeCV',outsample='2021-01-01',limit=None,smooth=avgSpace,cvType=min):
    """_summary_

    Args:
        title (str): name of strategy
        dbIndic (DataFrame): dataframe with indicators
        longIndicator (str):
        longAsset (str): asset on which we are long
        shortAsset (str, optional): asset on which we are short. Defaults to None.
        lags (range, optional): lags to test in grid search. Defaults to range(1,20).
        windows (range, optional): windows to test in grid search. Defaults to range(1,25,1).
        eps (list, optional): range of values for smoothing. Defaults to [2,1].
        target (str, optional): metric used to find best parameters. Defaults to 'sharpeCV'.
        outsample (str, optional): Date when the outsample starts. Defaults to '2021-01-01'.
        limit (float, optional): maximum turnover daily. Defaults to None.
        smooth (function, optional): function used to smooth the grid search (either avgSpace or sharpeSpace). Defaults to avgSpace.
        cvType (function, optional): function used for cross validation (either min or mean). Defaults to min.
    """
    assetsW=createAssets(long=longAsset,short=shortAsset)
    indicator=createIndicator(long=longIndicator,short=shortIndicator)
    GSTrend(indicator=indicator,assetsW=assetsW,title=title,lags=lags,windows=windows,eps=eps,target=target,outsample=outsample,limit=limit,smooth=smooth,cvType=cvType)

# strategies

## config

In [ ]:
# exemple de config : 1. trader HRW Wheat en momentum des US 2. trader Wheat en mean rev du Canada
strats={
    'HRWWheat':{'longIndicator':'US','longAsset':'HRWWheat','shortAsset':None},
    'Wheat':{'longIndicator':'CA','longAsset':None,'shortAsset':'Wheat'}
}

## exec

In [ ]:
from tqdm import tqdm
from warnings import filterwarnings
filterwarnings('ignore')
for title in tqdm(strats.keys()):
    createStrat(title=title,
    dbIndic=macro.copy(),
    longIndicator=strats[title]['longIndicator'],
    longAsset=strats[title]['longAsset'],
    shortAsset=strats[title]['shortAsset'],
    lags=range(1,20),
    windows=range(1,25,1),
    eps=[2,1],
    target='sharpeCV',outsample='2021-07-01',limit=None,
    smooth=avgSpace,cvType=min
    )
# smooth --> avgSpace ou sharpeSpace
# cvType --> min ou mean